In [ ]:
''' This script is developed to study the evolution of the NPDFS 
    OF MOLECULAR CLOUDS for various initial conditons.
    
    
    Date Created : 20May 2019
    Author  : Sayantan
    
    Model : TPB16V2 
    Using the new TP_module
    This was adopted from the new objected oriented module developed by Sayantan
    
    Output: 
    
    Movie showing the evolution of the PDF
    
    
'''

# import TP_module as mf
import glob
import numpy as np
import time as t
import os.path
import matplotlib.pyplot as plt
from TP_class_module import *
import multiprocessing as mp
from pylab import*

In [ ]:
Model = "TP_B25_v30"

current_directory = os.getcwd() 
print("the current directory in", current_directory)

save_path1 = './time_evol_plots_'+ Model
save_path = './fit_params_'+ Model
# param_folder= current_directory + '/fit_params_'+ Model
image_folder= current_directory + '/time_evol_plots_'+ Model

# os.path.isdir(path)

## Making the directory for the putput files
try:  
#     os.makedirs(param_folder)
    os.makedirs(image_folder)
except OSError:  
    print ("Creation of the directory %s failed/ not needed as it alread exit")
else:  
    print ("Successfully created the directory %s")

In [ ]:
list_of_files = glob.glob('../evolution_data/'+ Model +'/*.dat')
list = np.sort(list_of_files)
bin = [50,100,150,200,250,295] 
b = bin[1]

In [ ]:
def do_plot(number):
    fileName = list[number]
    fig = plt.figure(number)
    
## for compiling in series for the for-loop and comment the fucntion ###
# for fileName in list[123:150]:    
    print(fileName)
    
### To check the intial data as a plot and to get the xdata and the ydata ###
    MODEL = PDF_analysis(Model, fileName, binsize=150, upperlimit=None)    
    xdata1, ydata1, ydata_not_normalized, norm_const = MODEL.fittingdata()
#     fig = MODEL.plothist()
#     plt.show()    
    
    #     ## to determine the time
    time_step = int(fileName.replace('../evolution_data/'+ Model +'/colden_ev', '').replace('.dat', ''))
    time_in_my = str((np.float(time_step)-10)*2.5*10**(-2))
#     print(time_step)

    ### Model Selection 
    MODEL_FIT_LG = Fitting_the_PDF(Model, fileName, binsize=b,function_to_fit="Lognormal", bounds=((0, 0), (1, 1)))
    MODEL_FIT_PW = Fitting_the_PDF(Model, fileName, binsize=b,function_to_fit="Piecewise", bounds=((0, 0, -7, .5), (1, 1, -2, .7)))
     
    popt_lg, pcov_lg = MODEL_FIT_LG.model_fitting_parameters()
    

    popt, pcov =  MODEL_FIT_PW.model_fitting_parameters()
#     print(r'$\mu$=%5.2f, $\sigma$=%5.2f, $\alpha$=%5.2f, $\eta_{\rm {TP}}$=%5.2f' % tuple(popt))

    ## To get the intial chi quare values for both lognormal and piecewise function it    
    chisq_lg=MODEL_FIT_LG.get_chisquare_value(Best_fit_values=popt_lg)
    chisq_lgpw=MODEL_FIT_PW.get_chisquare_value(Best_fit_values=popt)
#     print("chisq lognormal+powerlaw (LC method)=", chisq_lgpw)
#     print("chisq lognormal(LC method)=",chisq_lg )
    
   
    
    if (chisq_lgpw <= 0.8*chisq_lg or chisq_lg >20 or chisq_lgpw > 20) and popt[2]>-5 and popt[3]< np.max(xdata1):
        
        fig = MODEL_FIT_PW.get_fitted_plot(time_in_my=time_in_my, Method='PYMC',guess_value_for_pymc=popt)
#         plt.show()
    else:        
        fig = MODEL_FIT_LG.get_fitted_plot(time_in_my=time_in_my)
        
    outputfilename_figfile = os.path.join(save_path1,'ev_'+str(time_step)+".jpg")
    plt.savefig(outputfilename_figfile,format='jpg',dpi=300)  
    plt.close()
    

In [ ]:
number_of_timestep = len(list)
# start_point = 
# end_point =
print(number_of_timestep)

# number_of_orbit = 10
start = time.time()
print("Plotting now...")
if __name__ == '__main__':
    pool = mp.Pool()
    pool.map(do_plot, range(0,number_of_timestep,1))
print("End of simulation")    
end = t.time()
T = end-start
print("Total time for parallel run %.2f"%T, 'sec')

In [ ]:
import cv2
print("Making the Video")

image_folder = save_path1 
video_name = 'video_'+ Model +'.mov'
if os.path.isfile(video_name):
    print("Removing the old video and making the new one")
    os.remove(video_name)

image = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
## sorting the images in ascending order of time
images= sorted(image, key=lambda x: int(x.replace("ev_", '').replace('.jpg', '')))
# images =sorted(image)
# print(images)
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, -1, 13, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

print("The video is done")

In [ ]:
## This is delete all the images to save space
fileList = os.listdir(save_path1)

for fileName in fileList:
#     print(fileName)
    os.remove(save_path1+"/"+fileName)